In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from numpy.linalg import inv
import ipyvolume as ipv
import math
import re
import random
import pdb
import json

# New Tactic

In [2]:
color_map = {
    'r':'red',
    'b':'blue',
    'g':'green',
    'w':'black',
    'y':'yellow',
    'o':'orange',
}

cube_match_res = {
    'x':re.compile('p..'),
    'y':re.compile('.n.'),
    'z':re.compile('..n'),
}

rotation_matrices={
    
    '+x':[
        [+1, +0 ,+0],
        [+0, +0, -1],
        [+0, +1, +0],
    ],
    
    '-x':[
        [+1, +0 ,+0],
        [+0, +0, +1],
        [+0, -1, +0],
    ],
    
    '+y':[
        [+0, +0, +1],
        [+0, +1, +0],
        [-1, +0, +0],
    ],
    
    '-y':[
        [+0, +0, -1],
        [+0, +1, +0],
        [+1, +0, +0],
    ],
    
    '+z':[
        [+0, -1, +0],
        [+1, +0, +0],
        [+0, +0, +1],
    ],
    
    '-z':[
        [+0, +1, +0],
        [-1, +0, +0],
        [+0, +0, +1],
    ],
    
    '+i':[
        [+1, +0, +0],
        [+0, +1, +0],
        [+0, +0, +1],
    ],
}


In [3]:
possibilities = ['+x', '-x', '+y', '-y', '+z', '-z', '+i']

In [4]:
cum_solution_matrices = {}

_temp = []

for rotation in possibilities :
        
    state = np.array(rotation_matrices['+i'])

    state = np.matmul(
        np.array(rotation_matrices[rotation]), 
        state,
    )

    _temp += [(state, rotation)]

    cum_solution_matrices[1] = _temp

for x in range(2,4):

    _temp = []
    
    combinations = pd.MultiIndex.from_product([possibilities for _ in range(x)]).tolist()
    
    for combination in combinations :
        
        state = np.array(rotation_matrices['+i'])

        for rotation in combination:

            state = np.matmul(
                np.array(rotation_matrices[rotation]), 
                state,
            )

        _temp += [(state, combination)]

    cum_solution_matrices[x] = _temp

In [5]:
pd.DataFrame(cum_solution_matrices[1])

,0,1
0,"[[1, 0, 0], [0, 0, -1], [0, 1, 0]]",+x
1,"[[1, 0, 0], [0, 0, 1], [0, -1, 0]]",-x
2,"[[0, 0, 1], [0, 1, 0], [-1, 0, 0]]",+y
3,"[[0, 0, -1], [0, 1, 0], [1, 0, 0]]",-y
4,"[[0, -1, 0], [1, 0, 0], [0, 0, 1]]",+z
5,"[[0, 1, 0], [-1, 0, 0], [0, 0, 1]]",-z
6,"[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",+i


In [6]:
def reset():
    
    cubes = []

    diagonals = []

    for x_dir in [-1, +1]:

        for y_dir in [-1, +1]:

            for z_dir in [-1, +1]:

                diagonals += [[x_dir, y_dir, z_dir]]

    for diagonal in diagonals:

        cubes += [{
            'vectors':[
                [diagonal[0], +0, +0],
                [+0, diagonal[1], +0],
                [+0, +0, diagonal[2]],
            ],
            'colors':[
                'g' if diagonal[0] > 0 else 'b',
                'y' if diagonal[1] > 0 else 'w',
                'r' if diagonal[2] > 0 else 'o',
            ],
            'cum_rotation_matrix':[
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1],
            ],
            'rotation_sequence':[],            
            
            'cum_solution_matrix':[
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1], 
            ],

            'solution_sequence':[],
        }]
        
    return cubes



In [7]:
def char_to_num(x):
    
    spread = 0.2
    
    if x == 'p':
    
        return +1*spread
    
    elif x == 'n':
    
        return -1*spread
    
    else:
        
        raise Exception('Incorrect string')
        
x = list(map(char_to_num, 'pnp'))

In [8]:
def num_to_char(x):
    
    if x > 0:
    
        return 'p'
    
    elif x < 0:
    
        return 'n'
    
    else:
        
        raise Exception('Incorrect num')

In [9]:
def get_quadrant(cube):

    sum_vector = np.sum(cube['vectors'], axis=0)
    
    return list(map(num_to_char, sum_vector))

In [10]:
def show_cube(*args, cubes, **kwargs):

    ipv.clear()

    ipv.pylab.xyzlim(-2,+2)
    
    for cube in cubes:

        quadrant = get_quadrant(cube)

        [x_beg, y_beg, z_beg] = list(map(char_to_num, quadrant))

        vectors  =cube['vectors']

        colors = cube['colors']

        for [x_dir, y_dir, z_dir], color in zip(vectors, colors):

#             ipv.pylab.plot(
#                 np.array([x_beg, x_beg + x_dir]), 
#                 np.array([y_beg, y_beg + y_dir]), 
#                 np.array([z_beg, z_beg + z_dir]),
#                 color=color_map[color],
#             )


            ipv.pylab.plot(
                np.array([x_beg*1.5, x_beg*1.5 + x_dir]), 
                np.array([y_beg*1.5, y_beg*1.5 + y_dir]), 
                np.array([z_beg*1.5, z_beg*1.5 + z_dir]),
                color=color_map[color],
            )
    
            ipv.pylab.scatter(
                np.array([x_beg*1.5 + x_dir]), 
                np.array([y_beg*1.5 + y_dir]), 
                np.array([z_beg*1.5 + z_dir]),
                color=color_map[color],
                marker='sphere',
                size=1,
            )

    ipv.view(20, 80)

    ipv.show()

In [11]:
def rotate_by_axis(*args, cubes, dimension, num_of_rotations, **kwargs):
    
    if num_of_rotations == 0:
        
        return cubes
        
    num_of_rotations = num_of_rotations % 4
    
    match_index_list=[]
    
    unmatched_index_list = []
    
    for index,cube in enumerate(cubes):
        
        quadrant_str = ''.join(get_quadrant(cube))
                               
        if cube_match_res[dimension[1]].match(quadrant_str):
                               
            match_index_list += [index]
            
        else:
            
            unmatched_index_list += [index]
                               
    assert len(match_index_list) == 4, 'get_positive_vectors did not find 4 vectors'
                         
    for index in match_index_list:        
                
        cubes[index]['vectors'] = np.matmul(
            np.array(rotation_matrices[dimension]), 
            np.array(cubes[index]['vectors']).T).T.tolist()     
        
        cubes[index]['cum_rotation_matrix'] = np.matmul(
            np.array(rotation_matrices[dimension]), 
            np.array(cubes[index]['cum_rotation_matrix']))
        
        cubes[index]['rotation_sequence'] += [dimension]
        
        
    for index in unmatched_index_list:        
        
        cubes[index]['rotation_sequence'] += ['+i']
        
    
    cubes = rotate_by_axis(cubes=cubes, dimension=dimension, num_of_rotations = num_of_rotations - 1)
    
    return cubes
                                               

In [12]:
test_cubes = reset()

original_cubes = reset()

In [13]:
show_cube(cubes=test_cubes); pd.DataFrame(test_cubes)

VBox(children=(Figure(anglex=1.3962634015954636, angley=0.3490658503988659, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([-0.3, -0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([-0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([-0.3, -0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([-0.3, -1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([-1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([0.3, 0.3]), z=array([-0.3, -0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([0.3]), z=array([-0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([0.3, 1.3]), z=array([-0.3, -0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([1.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([0.3, 0.3]), z=array([-0.3, -1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_se

,colors,cum_rotation_matrix,cum_solution_matrix,rotation_sequence,solution_sequence,vectors
0,"[b, w, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[-1, 0, 0], [0, -1, 0], [0, 0, -1]]"
1,"[b, w, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[-1, 0, 0], [0, -1, 0], [0, 0, 1]]"
2,"[b, y, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[-1, 0, 0], [0, 1, 0], [0, 0, -1]]"
3,"[b, y, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[-1, 0, 0], [0, 1, 0], [0, 0, 1]]"
4,"[g, w, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[1, 0, 0], [0, -1, 0], [0, 0, -1]]"
5,"[g, w, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[1, 0, 0], [0, -1, 0], [0, 0, 1]]"
6,"[g, y, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[1, 0, 0], [0, 1, 0], [0, 0, -1]]"
7,"[g, y, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[],"[[1, 0, 0], [0, 1, 0], [0, 0, 1]]"


In [14]:
#rotations = list(map(lambda x: ['x', 'y', 'z'][x], np.random.randint(low=0, high=3, size=10).tolist()))

rotations = ['+z', '+x']

In [15]:
for rotation in rotations:
    
    rotated_cubes = rotate_by_axis(
        cubes=test_cubes,
        dimension=rotation, 
        num_of_rotations=1
    )      

In [16]:
show_cube(cubes=test_cubes); pd.DataFrame(test_cubes)

VBox(children=(Figure(anglex=1.3962634015954636, angley=0.3490658503988659, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([0.3, 0.3]), z=array([-0.3, -1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([0.3]), z=array([-1.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 1.3]), y=array([0.3, 0.3]), z=array([-0.3, -0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([1.3]), y=array([0.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([0.3, 1.3]), z=array([-0.3, -0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([1.3]), z=array([-0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([-0.3, -0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([-0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([-0.3, -0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([-0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([-0.3, -1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=arr

,colors,cum_rotation_matrix,cum_solution_matrix,rotation_sequence,solution_sequence,vectors
0,"[b, w, o]","[[0, -1, 0], [0, 0, -1], [1, 0, 0]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+z, +x]",[],"[[0, 0, -1], [1, 0, 0], [0, 1, 0]]"
1,"[b, w, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+i, +i]",[],"[[-1, 0, 0], [0, -1, 0], [0, 0, 1]]"
2,"[b, y, o]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+z, +i]",[],"[[0, -1, 0], [-1, 0, 0], [0, 0, -1]]"
3,"[b, y, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+i, +i]",[],"[[-1, 0, 0], [0, 1, 0], [0, 0, 1]]"
4,"[g, w, o]","[[0, -1, 0], [0, 0, -1], [1, 0, 0]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+z, +x]",[],"[[0, 0, 1], [1, 0, 0], [0, 1, 0]]"
5,"[g, w, r]","[[1, 0, 0], [0, 0, -1], [0, 1, 0]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+i, +x]",[],"[[1, 0, 0], [0, 0, -1], [0, -1, 0]]"
6,"[g, y, o]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+z, +i]",[],"[[0, 1, 0], [-1, 0, 0], [0, 0, -1]]"
7,"[g, y, r]","[[1, 0, 0], [0, 0, -1], [0, 1, 0]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+i, +x]",[],"[[1, 0, 0], [0, 0, 1], [0, -1, 0]]"


# Examples

## starting and endings points for cube #1

In [17]:
beg = np.array([[-1, 0, 0], [0, -1, 0], [0, 0, -1]])
end = np.array([[0, 0, -1], [1, 0, 0], [0, 1, 0]])
trans_fwd = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]])

## get forward rotation matrix using matrix math

In [18]:
np.matmul(
    end.T,
    inv(beg.T)
).astype(int)

array([[ 0, -1,  0],
       [ 0,  0, -1],
       [ 1,  0,  0]])

## check if the data from the cum_rotation_matrix worked

In [19]:
np.matmul(
    trans_fwd,
    beg.T
).T

array([[ 0,  0, -1],
       [ 1,  0,  0],
       [ 0,  1,  0]])

## get the reverse rotation matrix - going from end to the beginning

In [20]:
np.matmul(
    beg.T,
    inv(end.T)
).astype(int)

array([[ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0]])

## just invert the forward rotation matrix - the answer should be the same

In [21]:
inv(trans_fwd).astype(int)

array([[ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0]])

# finally use the actual rotations documented

In [22]:
trans = np.identity(3); beg

for rotation in ['-x', '-z']:

    trans = np.matmul(rotation_matrices[rotation], trans)
    
trans.astype(int)

array([[ 0,  0,  1],
       [-1,  0,  0],
       [ 0, -1,  0]])

# Use matrix math to solve rotation

In [23]:
def get_unsolved_cubes():

    unsolved_cubes = [
        {
            'colors':['g', 'y', 'o'],
            'vectors':[[+0, -1, +0], [+0, +0, +1], [+1, +0, +0]],
        },
        {
            'colors':['w', 'b', 'r'],
            'vectors':[[+0, -1, 0], [-1, +0, +0], [+0, +0, +1]],
        },  
        {
            'colors':['r', 'w', 'g'],
            'vectors':[[+0, -1, +0], [-1, +0, +0], [+0, +0, -1]],
        },
        {
            'colors':['g', 'r', 'y'],
            'vectors':[[+1, +0, +0], [+0, +0, -1], [+0, -1, +0]],
        },    
        {
            'colors':['o', 'b', 'w'],
            'vectors':[[+0, +1, +0], [+0, +0, +1], [-1, +0, +0]],
        },
        {
            'colors':['b', 'r', 'y'],
            'vectors':[[+0, +0, +1], [+1, +0, +0], [+0, +1, +0]],
        },
        {
            'colors':['o', 'y', 'b'],
            'vectors':[[+0, +1, +0], [+0, +0, -1], [+1, +0, +0]],
        },
        {
            'colors':['w', 'o', 'g'],
            'vectors':[[+0, +1, +0], [+0, +0, -1], [-1, +0, +0]],
        },

    ]

    for cube in unsolved_cubes:

        cube['cum_solution_matrix']=[
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1], 
        ]

        cube['solution_sequence'] = []

        cube['cum_rotation_matrix'] = [
                    [1, 0, 0],
                    [0, 1, 0],
                    [0, 0, 1],

        ]
            
        cube['rotation_sequence'] = []
        
        
    return unsolved_cubes


In [24]:
def check_cube(*args, cubes, **kwargs):

    sum_vector = np.array([0, 0, 0])

    for cube in get_unsolved_cubes():

        for vector in cube['vectors']:

            sum_vector = sum_vector + np.array(vector)        

    return sum_vector == np.array([0, 0, 0])

In [25]:
def get_cube(*args, cubes, colors, **kwargs):
    
    requested_cube = sorted(colors)
    
    for index, attributes in enumerate(cubes):
        
        if sorted(attributes['colors']) == requested_cube:
            
            indices = list(map(attributes['colors'].index, requested_cube))
            
            return {'colors':requested_cube, 'vectors':[attributes['vectors'][i] for i in indices]}
    

In [26]:
def get_cum_rotation_matrix(*args, beg_cube, end_cube, **kwargs):
    
    if beg_cube['colors'] != end_cube['colors']:
        
        raise Exception('Colors sequence do not match')
        
    beg = beg_cube['vectors']
    
    end = end_cube['vectors']
  
    matrix = np.matmul(
        np.array(end).T,
        inv(np.array(beg).T)).astype(int)  

    return matrix    

In [27]:
def get_possible_rotation_sequence(*args, cum_solution_matrix, solution_matrices, **kwargs):
    
    rotations = []
    
    for x in range(1,4):

        for index, (possible_match, rotation) in enumerate(solution_matrices[x]):

            if (possible_match==cum_solution_matrix).flatten().sum()==9:

                rotations += [rotation]

        if len(rotations) > 0:
            
            return rotations
        
    return rotations



In [28]:
def update_solution_matrix_and_sequence(*args, unsolved_cubes, solved_cubes, **kwargs):

    for cube in unsolved_cubes:

        beg_cube = get_cube(cubes=solved_cubes, colors=cube['colors'])

        end_cube = get_cube(cubes=unsolved_cubes, colors=cube['colors'])

        cube['cum_solution_matrix'] = get_cum_rotation_matrix(
            beg_cube=end_cube, 
            end_cube=beg_cube, 
        )
        
        rotation_sequence = get_possible_rotation_sequence(
            cum_solution_matrix=cube['cum_solution_matrix'],
            solution_matrices=cum_solution_matrices
        )

        cube['solution_sequence'] = rotation_sequence
        
    return unsolved_cubes

In [29]:
def get_quadrant_re_for_cube_and_dimension(*args, cube, dimension, **kwargs):
    
    quadrant_str = get_quadrant(cube)
        
    assert dimension[1] in ['x', 'y', 'z'], "incorrect "

    if dimension[1] == 'x':
        
        return re.compile('{}..'.format(quadrant_str[0]))
    
    elif dimension[1] == 'y':
        
        return re.compile('.{}.'.format(quadrant_str[1]))
    
    elif dimension[1] == 'z':
        
        return re.compile('..{}'.format(quadrant_str[2])) 

    

In [30]:
def rotate_by_axis_2(*args, cubes, target_cube_index, dimension, num_of_rotations, **kwargs):

    if num_of_rotations == 0:
        
        return cubes
        
    num_of_rotations = num_of_rotations % 4
    
    match_index_list=[]
    
    unmatched_index_list = []
    
    cube_match_re = get_quadrant_re_for_cube_and_dimension(cube=cubes[target_cube_index], dimension=dimension)
       
    for index, cube in enumerate(cubes):
        
        quadrant_str = ''.join(get_quadrant(cube))
                               
        if cube_match_re.match(quadrant_str):
                               
            match_index_list += [index]
            
        else:
            
            unmatched_index_list += [index]
            
    assert len(match_index_list) == 4, 'get_positive_vectors did not find 4 vectors'
                         
    for index in match_index_list:        
                
        cubes[index]['vectors'] = np.matmul(
            np.array(rotation_matrices[dimension]), 
            np.array(cubes[index]['vectors']).T).T.tolist()     
        
        cubes[index]['cum_rotation_matrix'] = np.matmul(
            np.array(rotation_matrices[dimension]), 
            np.array(cubes[index]['cum_rotation_matrix']))
        
        cubes[index]['rotation_sequence'] += [dimension]
        
        
    for index in unmatched_index_list:        
        
        cubes[index]['rotation_sequence'] += ['+i']
        
    
    cubes = rotate_by_axis(cubes=cubes, dimension=dimension, num_of_rotations = num_of_rotations - 1)
    
    return cubes
            
    


In [31]:
def save_cube(*args, cubes, file_name, **kwargs):
    
    file_pointer = open(file_name, 'w')
    
    cubes = [{'colors':cube['colors'], 'vectors':cube['vectors']} for cube in cubes]
    
    json.dump(cubes, file_pointer)
    
    file_pointer.close()

In [32]:
def load_cube(*args, file_name, **kwargs):
    
    file_pointer = open(file_name, 'r')
    
    cubes = json.load(file_pointer)
    
    for cube in cubes:

        cube['cum_solution_matrix']=[
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1], 
        ]

        cube['solution_sequence'] = []

        cube['cum_rotation_matrix'] = [
                    [1, 0, 0],
                    [0, 1, 0],
                    [0, 0, 1],

        ]
            
        cube['rotation_sequence'] = []
        
        
    return cubes

In [33]:
unsolved_cubes = get_unsolved_cubes(); check_cube(cubes=unsolved_cubes)

array([ True,  True,  True])

In [34]:
show_cube(cubes=unsolved_cubes)

VBox(children=(Figure(anglex=1.3962634015954636, angley=0.3490658503988659, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color=array('green', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('green', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color=array('yellow', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([0.3, 1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('orange', dtype='<U6'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([0.3, 0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([0.3, 0.3])), Scatter(color=array('blue', dtype='<U4'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([0.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([0.3, 1.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([1.3])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -1.3]), z=array([-0.3, -0.3])), Scatter(color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-1.3]), z=array([-0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -1.3]), y=array([-0.3, -0.3]), z=array([-0.3, -0.3])), Scatter(color=array('black', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-1.3]), y=array([-0.3]), z=array([-0.3])), Scatter(color=array('green', dtype='<U5'), color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([-0.3, -0.3]), y=array([-0.3, -0.3]), z=array([-0.3, -1.3])), Scatter(color=array('green', dtype='<U5'), color_selected=array('white', dtype='<U5'), geo='sphere', size=array(1), size_selected=array(2.6), x=array([-0.3]), y=array([-0.3]), z=array([-1.3])), Scatter

## Start trying to solve the unsolved cube

In [35]:
unsolved_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=unsolved_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(unsolved_cubes)

,colors,cum_rotation_matrix,cum_solution_matrix,rotation_sequence,solution_sequence,vectors
0,"[g, y, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, -1, 0], [0, 0, 1], [-1, 0, 0]]",[],"[(-x, +y), (+y, +z), (+z, -x)]","[[0, -1, 0], [0, 0, 1], [1, 0, 0]]"
1,"[w, b, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[+i],"[[0, -1, 0], [-1, 0, 0], [0, 0, 1]]"
2,"[r, w, g]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]",[],"[(-x, +z), (-y, -x), (+z, -y)]","[[0, -1, 0], [-1, 0, 0], [0, 0, -1]]"
3,"[g, r, y]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, -1, 0], [0, 0, -1]]",[],"[(+x, +x), (-x, -x)]","[[1, 0, 0], [0, 0, -1], [0, -1, 0]]"
4,"[o, b, w]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]",[],"[(-x, +z), (-y, -x), (+z, -y)]","[[0, 1, 0], [0, 0, 1], [-1, 0, 0]]"
5,"[b, r, y]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [0, 1, 0], [1, 0, 0]]",[],[-y],"[[0, 0, 1], [1, 0, 0], [0, 1, 0]]"
6,"[o, y, b]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[-1, 0, 0], [0, 0, -1], [0, -1, 0]]",[],"[(+x, +y, +y), (+x, -y, -y), (-x, +z, +z), (-x...","[[0, 1, 0], [0, 0, -1], [1, 0, 0]]"
7,"[w, o, g]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[-1, 0, 0], [0, -1, 0], [0, 0, 1]]",[],"[(+z, +z), (-z, -z)]","[[0, 1, 0], [0, 0, -1], [-1, 0, 0]]"


In [36]:
unsolved_cubes = rotate_by_axis_2(cubes=unsolved_cubes, target_cube_index=0, dimension='-x', num_of_rotations=1)

In [37]:
unsolved_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=unsolved_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(unsolved_cubes)

,colors,cum_rotation_matrix,cum_solution_matrix,rotation_sequence,solution_sequence,vectors
0,"[g, y, o]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[0, 0, 1], [0, 1, 0], [-1, 0, 0]]",[-x],[+y],"[[0, 0, 1], [0, 1, 0], [1, 0, 0]]"
1,"[w, b, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[+i],[+i],"[[0, -1, 0], [-1, 0, 0], [0, 0, 1]]"
2,"[r, w, g]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]",[+i],"[(-x, +z), (-y, -x), (+z, -y)]","[[0, -1, 0], [-1, 0, 0], [0, 0, -1]]"
3,"[g, r, y]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]",[-x],[-x],"[[1, 0, 0], [0, -1, 0], [0, 0, 1]]"
4,"[o, b, w]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]",[+i],"[(-x, +z), (-y, -x), (+z, -y)]","[[0, 1, 0], [0, 0, 1], [-1, 0, 0]]"
5,"[b, r, y]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[0, -1, 0], [0, 0, -1], [1, 0, 0]]",[-x],"[(+x, -y), (-y, +z), (+z, +x)]","[[0, 1, 0], [1, 0, 0], [0, 0, -1]]"
6,"[o, y, b]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[-1, 0, 0], [0, -1, 0], [0, 0, 1]]",[-x],"[(+z, +z), (-z, -z)]","[[0, 0, -1], [0, -1, 0], [1, 0, 0]]"
7,"[w, o, g]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[-1, 0, 0], [0, -1, 0], [0, 0, 1]]",[+i],"[(+z, +z), (-z, -z)]","[[0, 1, 0], [0, 0, -1], [-1, 0, 0]]"


In [38]:
unsolved_cubes = rotate_by_axis_2(cubes=unsolved_cubes, target_cube_index=6, dimension='+z', num_of_rotations=1)

In [39]:
unsolved_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=unsolved_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(unsolved_cubes)

,colors,cum_rotation_matrix,cum_solution_matrix,rotation_sequence,solution_sequence,vectors
0,"[g, y, o]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[0, 0, 1], [0, 1, 0], [-1, 0, 0]]","[-x, +i]",[+y],"[[0, 0, 1], [0, 1, 0], [1, 0, 0]]"
1,"[w, b, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+i, +i]",[+i],"[[0, -1, 0], [-1, 0, 0], [0, 0, 1]]"
2,"[r, w, g]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[[0, 0, -1], [0, 1, 0], [1, 0, 0]]","[+i, +z]",[-y],"[[1, 0, 0], [0, -1, 0], [0, 0, -1]]"
3,"[g, r, y]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[-x, +i]",[-x],"[[1, 0, 0], [0, -1, 0], [0, 0, 1]]"
4,"[o, b, w]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","[+i, +i]","[(-x, +z), (-y, -x), (+z, -y)]","[[0, 1, 0], [0, 0, 1], [-1, 0, 0]]"
5,"[b, r, y]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","[[1, 0, 0], [0, 0, -1], [0, 1, 0]]","[-x, +z]",[+x],"[[-1, 0, 0], [0, 1, 0], [0, 0, -1]]"
6,"[o, y, b]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[-x, +z]",[+z],"[[0, 0, -1], [1, 0, 0], [0, 1, 0]]"
7,"[w, o, g]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[+i, +z]",[+z],"[[-1, 0, 0], [0, 0, -1], [0, -1, 0]]"


In [40]:
save_cube(cubes=unsolved_cubes, file_name='temp_1.json'); pd.DataFrame(unsolved_cubes)

,colors,cum_rotation_matrix,cum_solution_matrix,rotation_sequence,solution_sequence,vectors
0,"[g, y, o]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[0, 0, 1], [0, 1, 0], [-1, 0, 0]]","[-x, +i]",[+y],"[[0, 0, 1], [0, 1, 0], [1, 0, 0]]"
1,"[w, b, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[+i, +i]",[+i],"[[0, -1, 0], [-1, 0, 0], [0, 0, 1]]"
2,"[r, w, g]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[[0, 0, -1], [0, 1, 0], [1, 0, 0]]","[+i, +z]",[-y],"[[1, 0, 0], [0, -1, 0], [0, 0, -1]]"
3,"[g, r, y]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]","[-x, +i]",[-x],"[[1, 0, 0], [0, -1, 0], [0, 0, 1]]"
4,"[o, b, w]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","[+i, +i]","[(-x, +z), (-y, -x), (+z, -y)]","[[0, 1, 0], [0, 0, 1], [-1, 0, 0]]"
5,"[b, r, y]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","[[1, 0, 0], [0, 0, -1], [0, 1, 0]]","[-x, +z]",[+x],"[[-1, 0, 0], [0, 1, 0], [0, 0, -1]]"
6,"[o, y, b]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[-x, +z]",[+z],"[[0, 0, -1], [1, 0, 0], [0, 1, 0]]"
7,"[w, o, g]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]","[+i, +z]",[+z],"[[-1, 0, 0], [0, 0, -1], [0, -1, 0]]"


In [42]:
temp_cubes = load_cube(file_name='temp_1.json'); 

temp_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=temp_cubes, 
    solved_cubes=original_cubes
);

pd.DataFrame(temp_cubes)

,colors,cum_rotation_matrix,cum_solution_matrix,rotation_sequence,solution_sequence,vectors
0,"[g, y, o]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, 1], [0, 1, 0], [-1, 0, 0]]",[],[+y],"[[0, 0, 1], [0, 1, 0], [1, 0, 0]]"
1,"[w, b, r]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]",[],[+i],"[[0, -1, 0], [-1, 0, 0], [0, 0, 1]]"
2,"[r, w, g]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [0, 1, 0], [1, 0, 0]]",[],[-y],"[[1, 0, 0], [0, -1, 0], [0, 0, -1]]"
3,"[g, r, y]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 0, 1], [0, -1, 0]]",[],[-x],"[[1, 0, 0], [0, -1, 0], [0, 0, 1]]"
4,"[o, b, w]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, 0, -1], [1, 0, 0], [0, -1, 0]]",[],"[(-x, +z), (-y, -x), (+z, -y)]","[[0, 1, 0], [0, 0, 1], [-1, 0, 0]]"
5,"[b, r, y]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[1, 0, 0], [0, 0, -1], [0, 1, 0]]",[],[+x],"[[-1, 0, 0], [0, 1, 0], [0, 0, -1]]"
6,"[o, y, b]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]",[],[+z],"[[0, 0, -1], [1, 0, 0], [0, 1, 0]]"
7,"[w, o, g]","[[1, 0, 0], [0, 1, 0], [0, 0, 1]]","[[0, -1, 0], [1, 0, 0], [0, 0, 1]]",[],[+z],"[[-1, 0, 0], [0, 0, -1], [0, -1, 0]]"


In [ ]:
for cube in temp_cubes:
    
    for solution in cube['solution_sequence']

In [ ]:
unsolved_cubes = rotate_by_axis_2(cubes=unsolved_cubes, target_cube_index=7, dimension='+z', num_of_rotations=1)

In [ ]:
unsolved_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=unsolved_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(unsolved_cubes)

# potential next moves
target_cube_index=2, dimension='-y' > seemed promising

target_cube_index=3, dimension='+y' > also seems promising

target_cube_index=3, dimension='-y'

target_cube_index=0, dimension='-z' > maybe

In [ ]:
unsolved_cubes = rotate_by_axis_2(cubes=unsolved_cubes, target_cube_index=2, dimension='-y', num_of_rotations=1)

In [ ]:
unsolved_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=unsolved_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(unsolved_cubes)

In [ ]:
unsolved_cubes = rotate_by_axis_2(cubes=unsolved_cubes, target_cube_index=4, dimension='+z', num_of_rotations=1)

In [ ]:
unsolved_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=unsolved_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(unsolved_cubes)

# test cube

In [ ]:
target_cubes = test_cubes

In [ ]:
target_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=target_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(target_cubes)

In [ ]:
target_cubes = rotate_by_axis_2(cubes=target_cubes, target_cube_index=0, dimension='-x', num_of_rotations=1)

In [ ]:
target_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=target_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(target_cubes)

In [ ]:
target_cubes = rotate_by_axis_2(cubes=target_cubes, target_cube_index=0, dimension='-z', num_of_rotations=1)

In [ ]:
target_cubes = update_solution_matrix_and_sequence(
    unsolved_cubes=target_cubes, 
    solved_cubes=original_cubes
); pd.DataFrame(target_cubes)